# CCSD calculations

In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

np.set_printoptions(precision=3, linewidth=120)

## CCSD calculations on H2O // STO-3G

In this example, we will try to reproduce the [crawdad CCSD results](https://github.com/CrawfordGroup/ProgrammingProjects/tree/master/Project%2305).

In [2]:
molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz")
N = molecule.numberOfElectrons()
N_P = N // 2

print(molecule)

Number of electrons: 10
O  (0, -0.143226, 0)
H  (1.63804, 1.13655, -0)
H  (-1.63804, 1.13655, -0)



The first task is to generate the RHF reference, so we'll do an RHF SCF calculation.

In [3]:
r_spinor_basis = gqcpy.RSpinorBasis(molecule, "STO-3G")
K = r_spinor_basis.numberOfSpatialOrbitals()

S = r_spinor_basis.quantizeOverlapOperator().parameters()

In [4]:
sq_hamiltonian = gqcpy.SQHamiltonian.Molecular(r_spinor_basis, molecule)  # 'sq' for 'second-quantized'

The current Hamiltionian is expressed in the non-orthogonal AO basis. This is exactly what we need to start an RHF calculation.

In [5]:
environment = gqcpy.RHFSCFEnvironment.WithCoreGuess(N, sq_hamiltonian, S)
solver = gqcpy.RHFSCFSolver.DIIS()

objective = gqcpy.DiagonalRHFFockMatrixObjective(sq_hamiltonian)  # use the default threshold of 1.0e-08

Using this objective makes sure that the optimized expansion coefficients yield a diagonal Fock matrix, so they will represent the canonical RHF spin-orbital basis.

In [6]:
rhf_qc_structure = gqcpy.RHF.optimize(objective, solver, environment)

In [7]:
rhf_energy = rhf_qc_structure.groundStateEnergy() + gqcpy.Operator.NuclearRepulsion(molecule).value()
print(rhf_energy)

-74.94207992809503


This is very much in line with `crawdad`'s results: `-74.942079928192`.

In [8]:
rhf_parameters = rhf_qc_structure.groundStateParameters()
C = rhf_parameters.coefficientMatrix()

In [9]:
r_spinor_basis.transform(C)  # now represents a restricted spin-orbital basis using the canonical RHF MOs
print(r_spinor_basis.coefficientMatrix())

[[-9.944e-01 -2.392e-01 -1.015e-16 -9.368e-02 -7.673e-31 -1.116e-01  1.124e-15]
 [-2.410e-02  8.857e-01  3.043e-16  4.796e-01  4.009e-30  6.696e-01 -6.628e-15]
 [ 2.823e-18 -2.214e-16  6.073e-01  1.899e-16 -1.615e-16  8.910e-15  9.192e-01]
 [-3.162e-03  8.590e-02 -9.437e-16 -7.474e-01 -6.656e-30  7.385e-01 -6.356e-15]
 [ 7.102e-34  4.221e-32  2.040e-16 -7.595e-30  1.000e+00 -1.161e-30 -8.229e-17]
 [ 4.594e-03  1.440e-01  4.530e-01 -3.295e-01  1.532e-16 -7.098e-01 -7.325e-01]
 [ 4.594e-03  1.440e-01 -4.530e-01 -3.295e-01 -1.532e-16 -7.098e-01  7.325e-01]]


In [10]:
r_sq_hamiltonian = gqcpy.SQHamiltonian.Molecular(r_spinor_basis, molecule)
print(r_sq_hamiltonian.core().parameters())

[[-3.256e+01 -5.683e-01 -2.307e-16 -2.095e-01 -1.645e-30 -2.563e-01  2.645e-15]
 [-5.683e-01 -7.564e+00 -1.013e-15 -5.223e-01 -3.186e-30 -1.220e+00  1.277e-14]
 [-1.756e-16 -1.402e-15 -6.018e+00 -2.304e-15 -1.316e-15 -1.488e-14 -1.767e+00]
 [-2.095e-01 -5.223e-01 -2.304e-15 -6.610e+00 -1.948e-30  1.306e+00 -1.030e-14]
 [-1.671e-30 -3.359e-30 -1.316e-15 -1.880e-30 -7.347e+00  3.045e-29  1.622e-15]
 [-2.563e-01 -1.220e+00 -1.527e-14  1.306e+00  3.035e-29 -5.291e+00 -4.996e-15]
 [ 2.690e-15  1.326e-14 -1.767e+00 -1.044e-14  1.622e-15 -4.996e-15 -5.513e+00]]


Before continuing with the CCSD specifics, we must first prepare the molecular Hamiltonian in the correct spinor basis. Since we have implemented CCSD using general spinors, we should use a `GSpinorBasis`.

In [11]:
g_spinor_basis = gqcpy.GSpinorBasis.FromRestricted(r_spinor_basis)  # represents a general spinor basis, based off the restricted canonical RHF MOs
print(g_spinor_basis.coefficientMatrix())

[[-9.944e-01 -2.392e-01 -1.015e-16 -9.368e-02 -7.673e-31 -1.116e-01  1.124e-15  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [-2.410e-02  8.857e-01  3.043e-16  4.796e-01  4.009e-30  6.696e-01 -6.628e-15  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 2.823e-18 -2.214e-16  6.073e-01  1.899e-16 -1.615e-16  8.910e-15  9.192e-01  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [-3.162e-03  8.590e-02 -9.437e-16 -7.474e-01 -6.656e-30  7.385e-01 -6.356e-15  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 7.102e-34  4.221e-32  2.040e-16 -7.595e-30  1.000e+00 -1.161e-30 -8.229e-17  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 4.594e-03  1.440e-01  4.530e-01 -3.295e-01  1.532e-16 -7.098e-01 -7.325e-01  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 4.594e-03  1.440e-01 -4.530e-01 -3.295e-01 

We should note that the ordering of the MOs in this spinor basis is not by ascending energy, but they are ordered alphas first and then all betas.

For H2O, we have 10 electrons, so this means we shouldn't occupy the first 10 spinors (since that would correspond to occupying all alpha-spin-orbitals and then 3 beta-spin-orbitals), but we should occupy the first 5 alpha spin-orbitals and the first 5 beta-spin-orbitals.

This issue is circumvented by constructing an `OrbitalSpace`, which takes care of setting up the occupied and virtual orbital spaces. We don't have to construct this `OrbitalSpace` by hand: GQCP provides an API through the creation of the 'GHF' reference ONV.

In [12]:
M = g_spinor_basis.numberOfSpinors()
ghf_onv = gqcpy.SpinUnresolvedONV.GHF(M, N, rhf_parameters.spinOrbitalEnergiesBlocked())

orbital_space = ghf_onv.orbitalSpace()

In [13]:
print(orbital_space.description())

An orbital space with 14 orbitals.

	The occupied orbital indices are: 0 1 2 3 4 7 8 9 10 11 
	The active orbital indices are: 
	The virtual orbital indices are: 5 6 12 13 


CCSD is also implemented as a `QCMethod`. This means that we'll have to set up an environment and a particular solver.

In [14]:
sq_hamiltonian = gqcpy.SQHamiltonian.Molecular(g_spinor_basis, molecule)

In [15]:
solver = gqcpy.CCSDSolver.Plain()
environment = gqcpy.CCSDEnvironment.PerturbativeCCSD(sq_hamiltonian, orbital_space)

It's interesting to see that the initial, perturbative, T1 amplitudes are zero, which is a direct consequence of Brillouin's theorem: the occupied-virtual block of the Fock(ian) matrix is zero!

In [16]:
print(environment.t1_amplitudes[-1].asMatrix())

[[-5.697e-15 -6.325e-19 -0.000e+00 -0.000e+00]
 [-1.099e-13  4.264e-17 -0.000e+00 -0.000e+00]
 [-4.091e-16 -4.681e-13 -0.000e+00 -0.000e+00]
 [ 4.831e-13 -2.095e-16 -0.000e+00 -0.000e+00]
 [-5.365e-31  8.638e-18 -0.000e+00 -0.000e+00]
 [-0.000e+00 -0.000e+00 -5.698e-15 -6.325e-19]
 [-0.000e+00 -0.000e+00 -1.098e-13 -2.198e-16]
 [-0.000e+00 -0.000e+00 -3.685e-16 -4.688e-13]
 [-0.000e+00 -0.000e+00  4.834e-13 -2.637e-16]
 [-0.000e+00 -0.000e+00 -5.008e-31  8.638e-18]]


The initial T2-amplitudes are actually the MP2 T2-amplitudes, so the initial CCSD correlation energy is the MP2 correlation energy. Crawdad reports a value of `-0.049149636120`.

In [17]:
print(environment.electronic_energies[-1])

-0.049149636149434954


Let's now proceed by actually optimizing the CCSD wave function model. This may take a while if `gqcp` is built in Debug mode!

In [18]:
ccsd_qc_structure = gqcpy.CCSD.optimize(solver, environment)

Crawdad lists a converged CCSD energy as: `-0.070680088376`.

In [19]:
ccsd_correlation_energy = ccsd_qc_structure.groundStateEnergy()
print(ccsd_correlation_energy)

-0.07068008793712947
